In [1]:
cd ../..

/Users/hao/workspace/hpi_de/4th_Semester/Applied Machine Learning/sensor-data-gans


In [2]:
import numpy as np
import pandas as pd
import tensorflow
from sklearn.utils import shuffle
from utils.windowing import windowing_dataframe, transform_windows_df

## Read Data

In [3]:
train_df = pd.read_hdf('./datasets/mydata/train_df.h5')
val_df = pd.read_hdf('./datasets/mydata/val_df.h5')
test_df = pd.read_hdf('./datasets/mydata/test_df.h5')

## Windowing Timeseries Data

In [4]:
window_size = 5*50
step_size = int(window_size/2)
col_names = ['userAcceleration.x', 'userAcceleration.y', 'userAcceleration.z', 'userAcceleration.c']
method ='sliding'

In [5]:
train_windowed_df = windowing_dataframe(train_df, window_size=window_size, step_or_sample_size=step_size, col_names=col_names, method=method)
val_windowed_df = windowing_dataframe(val_df, window_size=window_size, step_or_sample_size=step_size, col_names=col_names, method=method)
test_windowed_df = windowing_dataframe(test_df, window_size=window_size, step_or_sample_size=step_size, col_names=col_names, method=method)

## Classifiers

### SVM

In [6]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report

In [7]:
input_cols = ['userAcceleration.c']
x_train, y_train = transform_windows_df(train_windowed_df, input_cols=input_cols, one_hot_encode=False, as_channel=False)
x_val, y_val = transform_windows_df(val_windowed_df, input_cols=input_cols, one_hot_encode=False, as_channel=False)
x_test, y_test = transform_windows_df(test_windowed_df, input_cols=input_cols, one_hot_encode=False, as_channel=False)

In [8]:
x_train = np.concatenate([x_train, x_val])
y_train = np.concatenate([y_train, y_val])

In [9]:
x_train = x_train.reshape((len(x_train), window_size))
x_test = x_test.reshape((len(x_test), window_size))

In [10]:
svm_clf = SVC()
svm_clf.fit(x_train, y_train)

SVC()

#### train performance

In [11]:
y_train_head = svm_clf.predict(x_train)

In [12]:
print(classification_report(y_train, y_train_head))

              precision    recall  f1-score   support

         0.0       0.62      1.00      0.77      1977
         1.0       0.87      0.46      0.61      2256
         2.0       1.00      0.82      0.90       829

    accuracy                           0.73      5062
   macro avg       0.83      0.76      0.76      5062
weighted avg       0.80      0.73      0.72      5062



#### test performance

In [13]:
y_test_head = svm_clf.predict(x_test)

In [14]:
print(classification_report(y_test, y_test_head))

              precision    recall  f1-score   support

         0.0       0.69      1.00      0.82       403
         1.0       0.78      0.35      0.49       387
         2.0       0.67      0.78      0.72       173

    accuracy                           0.70       963
   macro avg       0.71      0.71      0.67       963
weighted avg       0.72      0.70      0.67       963



### CNN

In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, MaxPool1D, GlobalMaxPool1D, Dropout

In [16]:
input_cols = ['userAcceleration.x', 'userAcceleration.y', 'userAcceleration.z']
x_train, y_train = transform_windows_df(train_windowed_df, input_cols=input_cols, one_hot_encode=True, as_channel=False)
x_val, y_val = transform_windows_df(val_windowed_df, input_cols=input_cols, one_hot_encode=True, as_channel=False)
x_test, y_test = transform_windows_df(test_windowed_df, input_cols=input_cols, one_hot_encode=True, as_channel=False)

In [20]:
input_shape = x_train[0].shape

### build neural net

In [45]:
model = Sequential()
model.add(Conv1D(32, kernel_size=5, activation='relu', input_shape=input_shape))
model.add(MaxPool1D(pool_size=2))
model.add(Conv1D(16, kernel_size=5, activation='relu'))
model.add(MaxPool1D(pool_size=2))
model.add(GlobalMaxPool1D())
model.add(Dense(8, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_16 (Conv1D)           (None, 246, 32)           512       
_________________________________________________________________
max_pooling1d_11 (MaxPooling (None, 123, 32)           0         
_________________________________________________________________
conv1d_17 (Conv1D)           (None, 119, 16)           2576      
_________________________________________________________________
max_pooling1d_12 (MaxPooling (None, 59, 16)            0         
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 16)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 8)                 136       
_________________________________________________________________
dense_11 (Dense)             (None, 3)               

In [46]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [47]:
x, y = shuffle(x_train, y_train)
model.fit(x, y, batch_size=32, epochs=10, validation_data=(x_val, y_val))

Epoch 1/10
126/126 [==============================] - 1s 8ms/step - loss: 0.8219 - accuracy: 0.7417 - val_loss: 0.5683 - val_accuracy: 0.9556
Epoch 2/10
126/126 [==============================] - 1s 7ms/step - loss: 0.4195 - accuracy: 0.9771 - val_loss: 0.3364 - val_accuracy: 0.9903
Epoch 3/10
126/126 [==============================] - 1s 7ms/step - loss: 0.1882 - accuracy: 0.9963 - val_loss: 0.0709 - val_accuracy: 0.9990
Epoch 4/10
126/126 [==============================] - 1s 7ms/step - loss: 0.0298 - accuracy: 0.9985 - val_loss: 0.0188 - val_accuracy: 0.9990
Epoch 5/10
126/126 [==============================] - 1s 8ms/step - loss: 0.0103 - accuracy: 0.9988 - val_loss: 0.0116 - val_accuracy: 0.9990
Epoch 6/10
126/126 [==============================] - 1s 7ms/step - loss: 0.0064 - accuracy: 0.9993 - val_loss: 0.0097 - val_accuracy: 0.9990
Epoch 7/10
126/126 [==============================] - 1s 7ms/step - loss: 0.0049 - accuracy: 0.9990 - val_loss: 0.0093 - val_accuracy: 0.9990
Epoch 

In [48]:
model.evaluate(x_test, y_test)

31/31 [==============================] - 0s 2ms/step - loss: 0.0187 - accuracy: 0.9917


[0.01867806911468506, 0.9916926026344299]